In this notebook, we will do some plots to estimate redshift results for 3 methods below:
1. bliss redshift estimate directly from images
2. redshift estimate based on GT mag 
3. redshift estimate based on LSST predicted flux

each plot should have metrics vs galaxy fluxes

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import torch
from os import environ
from pathlib import Path
from einops import rearrange
import pickle
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from hydra import initialize, compose
from hydra.utils import instantiate

from pytorch_lightning.utilities import move_data_to_device

from bliss.catalog import FullCatalog, BaseTileCatalog, TileCatalog
from bliss.surveys.dc2 import DC2, unsqueeze_tile_dict
from case_studies.redshift.evaluation.utils.load_lsst import get_lsst_full_cat
from case_studies.redshift.evaluation.utils.safe_metric_collection import SafeMetricCollection as MetricCollection
from case_studies.redshift.redshift_from_img.encoder.metrics import RedshiftMeanSquaredErrorBin

environ["BLISS_HOME"] = str(Path().resolve().parents[2])

output_dir = Path("./DC2output/")
output_dir.mkdir(parents=True, exist_ok=True)

# change this model path according to your training setting
model_path = "/home/qiaozhih/bliss/case_studies/redshift/redshift_from_img/DC2_redshift_training/DC2_redshift_only_bin/checkpoints/encoder_0.174964.ckpt"
lsst_root_dir = "/data/scratch/dc2_nfs/"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [3]:
with initialize(config_path=".", version_base=None):
    notebook_cfg = compose("notebook_plot")

In [4]:
# set up testing dataset
dataset = instantiate(notebook_cfg.train.data_source)
dataset.setup()


In [5]:
import os
os.environ['OMP_NUM_THREADS'] = '16'
os.environ['MKL_NUM_THREADS'] = '16'
os.environ['NUMEXPR_NUM_THREADS'] = '16'

# example of test dataset

In [9]:
len(dataset.image_ids())

245000

In [10]:
len(dataset.test_dataset)

24500

### 1. bliss redshift estimate directly from images

In [6]:
# Loop through the test set and update the metric
# load bliss trained model
bliss_encoder = instantiate(notebook_cfg.encoder).to(device=device)
pretrained_weights = torch.load(model_path, device)["state_dict"]
bliss_encoder.load_state_dict(pretrained_weights)
bliss_encoder.eval();
# for batch_idx, batch in tqdm(enumerate(dataset.test_dataloader()), total=len(dataset.test_dataloader())):
#     batch["images"] = batch["images"].to(device)
#     bliss_encoder.update_metrics(batch, batch_idx)

# bliss_out_dict = bliss_encoder.mode_metrics.compute()

In [7]:
bliss_output_path = output_dir / "bliss_output_all.pkl"

if not bliss_output_path.exists():
    for batch_idx, batch in tqdm(enumerate(dataset.test_dataloader()), total=len(dataset.test_dataloader())):
        batch["images"] = batch["images"].to(device)
        bliss_encoder.update_metrics(batch, batch_idx)
    bliss_out_dict = bliss_encoder.mode_metrics.compute()

    with open(bliss_output_path, "wb") as outp:  # Overwrites any existing file.
        pickle.dump(bliss_out_dict, outp, pickle.HIGHEST_PROTOCOL)
else:
    with open(bliss_output_path, "rb") as inputp:
        bliss_out_dict = pickle.load(inputp)

In [ ]:
bliss_out_dict

In [ ]:
flux_ranges = ['[100-200]', '[200-400]', '[400-600]', '[600-1000]', '[>1000]']
mse_values = [bliss_out_dict[f'redshifts/mse_bin_{i}'] for i in range(5)]

plt.figure(figsize=(10, 6))
plt.plot(flux_ranges, mse_values)
plt.xlabel('Flux Range')
plt.ylabel('Mean Squared Error')
plt.title('Mean Squared Error per Flux Range')
plt.show()


In [ ]:
flux_ranges = ['[100-200]', '[200-400]', '[400-600]', '[600-1000]', '[>1000]']
bias_values = [bliss_out_dict[f'redshifts/bias_bin_{i}'] for i in range(5)]

plt.figure(figsize=(10, 6))
plt.plot(flux_ranges, bias_values)
plt.xlabel('Flux Range')
plt.ylabel('Bias')
plt.title('Bias per Flux Range')
plt.show()

In [ ]:
flux_ranges = ['[100-200]', '[200-400]', '[400-600]', '[600-1000]', '[>1000]']
nmad_values = [bliss_out_dict[f'redshifts/nmad_bin_{i}'] for i in range(5)]

plt.figure(figsize=(10, 6))
plt.plot(flux_ranges, nmad_values)
plt.xlabel('Flux Range')
plt.ylabel('NMAD')
plt.title('NMAD per Flux Range')
plt.show()

In [ ]:
flux_ranges = ['[100-200]', '[200-400]', '[400-600]', '[600-1000]', '[>1000]']
outlier_fraction_values = [bliss_out_dict[f'redshifts/outlier_fraction_bin_{i}'] for i in range(5)]

plt.figure(figsize=(10, 6))
plt.plot(flux_ranges, outlier_fraction_values)
plt.xlabel('Flux Range')
plt.ylabel('Outlier Fraction')
plt.title('Outlier Fraction per Flux Range')
plt.show()


In [ ]:
flux_ranges = ['[100-200]', '[200-400]', '[400-600]', '[600-1000]', '[>1000]']
outlier_fraction_cata_values = [bliss_out_dict[f'redshifts/outlier_fraction_cata_bin_{i}'] for i in range(5)]

plt.figure(figsize=(10, 6))
plt.plot(flux_ranges, outlier_fraction_cata_values)
plt.xlabel('Flux Range')
plt.ylabel('Outlier Fraction Catastrophic')
plt.title('Outlier Fraction Catastrophic per Flux Range')
plt.show()

### 2. redshift estimate based on GT mag

In [7]:
from case_studies.redshift.network_rs import PhotoZFromMag
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import torch.nn as nn

In [8]:
num_bins = 1
group_size = 128
batch_size = 1
in_dim = 6
device = "cuda"
path = "/data/scratch/qiaozhih/training_runs/00120-run/tensorboard_logs/version_0/checkpoints/reg_val_loss=0.000367_epoch=29.ckpt"
network_options = {
    "hidden_dim": 256,
    "out_dim": num_bins,
    "n_epochs": 50001,
    "outdir": "/home/qiaozhih/bliss/case_studies/redshift/training_runs/",
    "snap": 1,  # how many epoches to save one model once
    "loss_fcn": torch.nn.MSELoss(),  # loss func
    # 'loss_fcn': torch.nn.CrossEntropyLoss(),                     # loss func
    "dropout_rate": 0.5,
    "learning_rate": 1e-3,
    "group_size": group_size,
    "num_gpu": 1,
    "gpu_device": [0],
}

In [9]:
reg = PhotoZFromMag(
    in_dim,
    network_options["hidden_dim"],
    network_options["out_dim"],
    network_options["dropout_rate"],
    network_options["learning_rate"],
    network_options["loss_fcn"],
)
reg = reg.load_from_checkpoint(
    path,
    in_dim=in_dim,
    hidden_dim=network_options["hidden_dim"],
    out_dim=network_options["out_dim"],
    dropout_rate=network_options["dropout_rate"],
    learning_rate=network_options["learning_rate"],
    loss_fcn=network_options["loss_fcn"],
)
reg.to(device)
reg.eval()

PhotoZFromMag(
  (loss_fcn): MSELoss()
  (net): Sequential(
    (0): Linear(in_features=6, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=256, bias=True)
    (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=256, out_features=256, bias=True)
    (10): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [10]:
def est_redshifts_with_reg_change_to_cat(cat: TileCatalog, reg: PhotoZFromMag, tile_slen: int, min_flux_for_loss: int=50, reference_band: int=2) -> BaseTileCatalog | FullCatalog:
    """estimate redshift by inputting true mag using regressor, then convert them to Basetilecatalog

    Args:
        cat: catalog
        reg: regressor that accept ugrizy mag

    Returns:
        Catalog that containing pred redshifts
    """
    cat = cat.filter_tile_catalog_by_flux(
        min_flux=min_flux_for_loss,
        band=reference_band,
    )
    mag = cat.magnitudes_njy.to(device)
    new_order = [3, 0, 2, 1, 5, 4] # change giruyz to ugrizy
    mag = mag[..., new_order]
    is_galaxy = cat.galaxy_bools.to(device)
    N, H, W, M, _ = mag.shape
    mag = mag.reshape(-1, 6)
    is_galaxy = is_galaxy.reshape(-1, 1)
    pred_rs = torch.zeros_like(mag[..., 0])
    for i in range(len(mag)):
        if is_galaxy[i][0]: 
            x = mag[i].unsqueeze(0) # 1, 6
            a = reg.net(x) # 1, 1
            pred_rs[i] = reg.net(mag[i].unsqueeze(0)).squeeze(0)
    pred_rs = pred_rs.to(cat.device)
    pred_rs = {"redshifts": pred_rs.reshape((N, H, W, M, 1))}

    cat["redshifts"] = pred_rs["redshifts"]
    return cat

GT data preparation

In [ ]:
gt_output_path = output_dir / "gt_output_all.pkl"
if not gt_output_path.exists():
    metrics = instantiate(notebook_cfg.encoder.metrics)
    matcher = instantiate(notebook_cfg.encoder.matcher)
    metrics.reset()
    for batch_idx, batch in tqdm(enumerate(dataset.test_dataloader()), total=len(dataset.test_dataloader())):
        true_cat = TileCatalog(tile_slen=4, d=batch["tile_catalog"]) # TODO tile_slen
        est_cat = est_redshifts_with_reg_change_to_cat(true_cat, reg, tile_slen=4)# TODO
        matching = matcher.match_catalogs(true_cat, est_cat)
        metrics.update(true_cat, est_cat, matching)

    gt_out_dict = metrics.compute()

    with open(gt_output_path, "wb") as outp:  # Overwrites any existing file.
        pickle.dump(gt_out_dict, outp, pickle.HIGHEST_PROTOCOL)
else:
    with open(gt_output_path, "rb") as inputp:
        gt_out_dict = pickle.load(inputp)

In [ ]:
bliss_out_dict

In [ ]:
metrics = ['mse', 'nmad', 'outlier_fraction_cata', 'outlier_fraction', 'bias']
fig, ax = plt.subplots(len(metrics), figsize=(10, 30))
for i, metric in enumerate(metrics):
    flux_ranges = ['[100-200]', '[200-400]', '[400-600]', '[600-1000]', '[>1000]']
    gt_mse_values = [gt_out_dict[f'redshifts/{metric}_bin_{i}'] for i in range(5)]
    bliss_mse_values = [bliss_out_dict[f'redshifts/{metric}_bin_{i}'] for i in range(5)]

    ax[i].plot(flux_ranges, bliss_mse_values, label="bliss")
    ax[i].plot(flux_ranges, gt_mse_values, label="gt_reg")
    if metric != 'bias':
        ax[i].set_yscale("log")
    ax[i].set_xlabel('Flux Range')
    ax[i].set_ylabel(metric)
    ax[i].set_title(f'{metric} per Flux Range')
    ax[i].legend()
plt.show()

In [ ]:
gt_out_dict

### 3. redshift estimate based on LSST predicted flux

In [11]:
import GCRCatalogs
import pandas as pd
import torch

from bliss.catalog import FullCatalog, SourceType

# load lsst data
def get_lsst_catalog_tensors_dict(lsst_root_dir: str):
    GCRCatalogs.set_root_dir(lsst_root_dir)
    lsst_catalog_gcr = GCRCatalogs.load_catalog("desc_dc2_run2.2i_dr6_object_with_truth_match")
    lsst_catalog_sub = lsst_catalog_gcr.get_quantities(
        [
            "id_truth",
            "objectId",
            "ra",
            "dec",
            "truth_type",
            "cModelFlux_u",
            "cModelFluxErr_u",
            "cModelFlux_g",
            "cModelFluxErr_g",
            "cModelFlux_r",
            "cModelFluxErr_r",
            "cModelFlux_i",
            "cModelFluxErr_i",
            "cModelFlux_z",
            "cModelFluxErr_z",
            "cModelFlux_y",
            "cModelFluxErr_y",
        ]
    )
    lsst_catalog_df = pd.DataFrame(lsst_catalog_sub)
    lsst_flux_lst = [
        lsst_catalog_df["cModelFlux_g"],
        lsst_catalog_df["cModelFlux_i"],
        lsst_catalog_df["cModelFlux_r"],
        lsst_catalog_df["cModelFlux_u"],
        lsst_catalog_df["cModelFlux_y"],
        lsst_catalog_df["cModelFlux_z"],
    ]
    lsst_flux_tensors_lst = [torch.tensor(flux.values).view(-1, 1) for flux in lsst_flux_lst]
    return {
        "truth_type": torch.tensor(lsst_catalog_df["truth_type"].values).view(-1, 1),
        "flux": torch.cat(lsst_flux_tensors_lst, dim=1),
        "ra": torch.tensor(lsst_catalog_df["ra"].values),
        "dec": torch.tensor(lsst_catalog_df["dec"].values),
    }
    
def get_lsst_catalog_tensors_dict(lsst_root_dir: str):
    GCRCatalogs.set_root_dir(lsst_root_dir)
    lsst_catalog_gcr = GCRCatalogs.load_catalog("desc_dc2_run2.2i_dr6_object_with_truth_match")
    lsst_catalog_sub = lsst_catalog_gcr.get_quantities(
        [
            "id_truth",
            "objectId",
            "ra",
            "dec",
            "truth_type",
            "cModelFlux_u",
            "cModelFluxErr_u",
            "cModelFlux_g",
            "cModelFluxErr_g",
            "cModelFlux_r",
            "cModelFluxErr_r",
            "cModelFlux_i",
            "cModelFluxErr_i",
            "cModelFlux_z",
            "cModelFluxErr_z",
            "cModelFlux_y",
            "cModelFluxErr_y",
        ]
    )
    lsst_catalog_df = pd.DataFrame(lsst_catalog_sub)
    lsst_flux_lst = [
        lsst_catalog_df["cModelFlux_g"],
        lsst_catalog_df["cModelFlux_i"],
        lsst_catalog_df["cModelFlux_r"],
        lsst_catalog_df["cModelFlux_u"],
        lsst_catalog_df["cModelFlux_y"],
        lsst_catalog_df["cModelFlux_z"],
    ]
    lsst_flux_tensors_lst = [torch.tensor(flux.values).view(-1, 1) for flux in lsst_flux_lst]
    return {
        "truth_type": torch.tensor(lsst_catalog_df["truth_type"].values).view(-1, 1),
        "flux": torch.cat(lsst_flux_tensors_lst, dim=1),
        "ra": torch.tensor(lsst_catalog_df["ra"].values),
        "dec": torch.tensor(lsst_catalog_df["dec"].values),
    }


In [18]:
import torch

from bliss.catalog import FullCatalog, SourceType, TileCatalog
from bliss.surveys.dc2 import from_wcs_header_str_to_wcs

def concatenate_tile_dicts(tile_dict_list):
    output_tile_cat_dict = {}
    for k in tile_dict_list[0].keys():
        if k not in output_tile_cat_dict:
            output_tile_cat_dict[k] = []
        for tile_dict in tile_dict_list:
            output_tile_cat_dict[k].append(tile_dict[k])

    for k, v in output_tile_cat_dict.items():
        output_tile_cat_dict[k] = torch.cat(v, dim=0)

    return output_tile_cat_dict


class LSSTPredictor:
    def __init__(
        self, lsst_root_dir, r_band_min_flux, tile_slen, max_sources_per_tile, tiles_to_crop
    ) -> None:
        self.lsst_data = get_lsst_catalog_tensors_dict(lsst_root_dir)
        self.r_band_min_flux = r_band_min_flux
        self.tile_slen = tile_slen
        self.max_sources_per_tile = max_sources_per_tile
        self.tiles_to_crop = tiles_to_crop

        self.buffered_wcs_header_str = None
        self.buffered_lsst_plocs = None

    def _predict_one_image(self, wcs_header_str, image_lim, height_index, width_index):
        if wcs_header_str != self.buffered_wcs_header_str:
            lsst_ra = self.lsst_data["ra"]
            lsst_dec = self.lsst_data["dec"]
            cur_image_wcs = from_wcs_header_str_to_wcs(wcs_header_str)
            lsst_plocs = FullCatalog.plocs_from_ra_dec(lsst_ra, lsst_dec, cur_image_wcs)
            self.buffered_wcs_header_str = wcs_header_str
            self.buffered_lsst_plocs = lsst_plocs
        else:
            lsst_plocs = self.buffered_lsst_plocs
    
        lsst_source_type = self.lsst_data["truth_type"]
        lsst_flux = self.lsst_data["flux"]

        x0_mask = (lsst_plocs[:, 0] > height_index * image_lim) & (
            lsst_plocs[:, 0] < (height_index + 1) * image_lim
        )
        x1_mask = (lsst_plocs[:, 1] > width_index * image_lim) & (
            lsst_plocs[:, 1] < (width_index + 1) * image_lim
        )
        lsst_x_mask = x0_mask * x1_mask
        # filter r band
        filter_band = ["g", "i", "r", "u", "y", "z"]
        for i in range(len(filter_band)):
            lsst_flux_mask = (lsst_flux[:, i] > self.r_band_min_flux) & (lsst_flux[:, i] < torch.inf)
            lsst_x_mask = lsst_x_mask * lsst_flux_mask

        lsst_mask = lsst_x_mask
        lsst_plocs = lsst_plocs[lsst_mask, :] % image_lim
        lsst_source_type = torch.where(
            lsst_source_type[lsst_mask] == SourceType.STAR, SourceType.STAR, SourceType.GALAXY
        )
        lsst_flux = lsst_flux[lsst_mask, :]
        lsst_n_sources = torch.tensor([lsst_plocs.shape[0]])

        return FullCatalog(
            height=image_lim,
            width=image_lim,
            d={
                "plocs": lsst_plocs.unsqueeze(0),
                "n_sources": lsst_n_sources,
                "source_type": lsst_source_type.unsqueeze(0),
                "galaxy_fluxes": lsst_flux.unsqueeze(0),
                "star_fluxes": lsst_flux.unsqueeze(0).clone(),
            },
        ).to_tile_catalog(self.tile_slen, self.max_sources_per_tile, ignore_extra_sources=True)

    def predict(self, wcs_header_str_list, image_lim, height_index_list, width_index_list):
        assert len(wcs_header_str_list) == len(height_index_list), "unequal input list size"
        assert len(wcs_header_str_list) == len(width_index_list), "unequal input list size"
        tile_dict_list = []
        predict_input_data = zip(wcs_header_str_list, height_index_list, width_index_list)
        for wcs_header_str, height_index, width_index in predict_input_data:
            tile_dict_list.append(
                self._predict_one_image(wcs_header_str, image_lim, height_index, width_index).data
            )

        merged_tile_dict = concatenate_tile_dicts(tile_dict_list)
        return TileCatalog(self.tile_slen, merged_tile_dict)

In [19]:
lsst_output_path = output_dir / "lsst_output_all.pkl"
if not lsst_output_path.exists():
    metrics = instantiate(notebook_cfg.encoder.metrics)
    matcher = instantiate(notebook_cfg.encoder.matcher)
    metrics.reset()
    # args
    lsst_root_dir = "/data/scratch/dc2_nfs/"
    image_lim = 80
    r_band_min_flux = 50
    tile_slen = 4
    max_sources_per_tile = 1
    tiles_to_crop = 1

    lsst_predictor = LSSTPredictor(lsst_root_dir, r_band_min_flux, tile_slen, max_sources_per_tile, tiles_to_crop)
    for batch_idx, batch in tqdm(enumerate(dataset.val_dataloader()), total=len(dataset.test_dataloader())):
        true_cat = TileCatalog(tile_slen=4, d=batch["tile_catalog"])
        lsst_cat = lsst_predictor.predict(batch["wcs_header_str"], image_lim, batch["image_height_index"], batch["image_width_index"])
        lsst_cat = lsst_cat.to_full_catalog()
        est_cat = est_redshifts_with_reg_change_to_cat(lsst_cat, reg, tile_slen=4)
        matching = matcher.match_catalogs(true_cat, est_cat)
        metrics.update(true_cat, est_cat, matching)

    lsst_out_dict = metrics.compute()

    with open(lsst_output_path, "wb") as outp:  # Overwrites any existing file.
        pickle.dump(lsst_out_dict, outp, pickle.HIGHEST_PROTOCOL)
else:
    with open(lsst_output_path, "rb") as inputp:
        lsst_out_dict = pickle.load(inputp)

In [20]:

# metrics = instantiate(notebook_cfg.encoder.metrics)
# matcher = instantiate(notebook_cfg.encoder.matcher)
# metrics.reset()
# # args
# lsst_root_dir = "/data/scratch/dc2_nfs/"
# image_lim = 80
# r_band_min_flux = 50
# tile_slen = 4
# max_sources_per_tile = 1
# tiles_to_crop = 1

# lsst_predictor = LSSTPredictor(lsst_root_dir, r_band_min_flux, tile_slen, max_sources_per_tile, tiles_to_crop)
# for batch_idx, batch in tqdm(enumerate(dataset.val_dataloader()), total=len(dataset.test_dataloader())):
#     if batch_idx >= 5:
#         true_cat = TileCatalog(tile_slen=4, d=batch["tile_catalog"])
#         lsst_cat = lsst_predictor.predict(batch["wcs_header_str"], image_lim, batch["image_height_index"], batch["image_width_index"])
#         est_cat = est_redshifts_with_reg_change_to_cat(lsst_cat, reg, tile_slen=4)
#         est_cat = est_cat.to_full_catalog()
#         true_cat = true_cat.to_full_catalog()
#         matching = matcher.match_catalogs(true_cat, est_cat)
#         metrics.update(true_cat, est_cat, matching)
#         print(metrics.compute())

# lsst_out_dict = metrics.compute()


  2%|▏         | 6/383 [04:12<6:42:31, 64.06s/it]

total num of pts: tensor([1220.])
{'redshifts/mse': 0.2829994857311249}


  2%|▏         | 7/383 [07:38<10:38:29, 101.89s/it]

total num of pts: tensor([1405.])
{'redshifts/mse': 0.2839999794960022}


  2%|▏         | 8/383 [11:27<14:15:18, 136.85s/it]

...

100%|█████████▉| 382/383 [17:12:34<02:37, 157.55s/it]

total num of pts: tensor([74138.])
{'redshifts/mse': 0.2848495543003082}


100%|██████████| 383/383 [17:14:40<00:00, 147.94s/it]

total num of pts: tensor([74290.])
{'redshifts/mse': 0.2848811149597168}


100%|██████████| 383/383 [17:14:40<00:00, 162.09s/it]


In [22]:
lsst_out_dict

{'redshifts/mse': 0.2848811149597168}

# LSST flux using table and Reg

In [27]:
# load lsst
GCRCatalogs.set_root_dir(lsst_root_dir)
lsst_catalog_gcr = GCRCatalogs.load_catalog("desc_dc2_run2.2i_dr6_object_with_truth_match")
lsst_catalog_sub = lsst_catalog_gcr.get_quantities(
    [
        "id_truth",
        "objectId",
        "ra",
        "dec",
        "truth_type",
        "cModelFlux_u",
        "cModelFluxErr_u",
        "cModelFlux_g",
        "cModelFluxErr_g",
        "cModelFlux_r",
        "cModelFluxErr_r",
        "cModelFlux_i",
        "cModelFluxErr_i",
        "cModelFlux_z",
        "cModelFluxErr_z",
        "cModelFlux_y",
        "cModelFluxErr_y",
    ]
)
lsst_catalog_df = pd.DataFrame(lsst_catalog_sub)

In [28]:
lsst_catalog_df

,cModelFlux_u,cModelFluxErr_i,cModelFlux_i,cModelFlux_r,cModelFluxErr_z,objectId,cModelFlux_z,cModelFlux_y,ra,cModelFluxErr_g,truth_type,cModelFluxErr_u,cModelFluxErr_r,cModelFluxErr_y,cModelFlux_g,dec,id_truth
0,146.454617,34.417397,348.178418,292.783253,97.712447,15982501021353368,494.325910,1082.446667,56.851861,16.475798,1,46.146050,19.407432,181.637104,304.932765,-38.677585,8757401501
1,78.795684,23.461088,135.422623,52.530998,66.623090,15982501021353393,-42.737415,49.059334,56.799465,11.117315,1,31.512319,12.540611,141.247208,92.324142,-38.677672,8750694182
2,612.543642,30.441242,610.091646,546.735183,79.490015,15982501021353397,602.223260,528.330959,56.883249,14.003504,1,38.914347,16.139224,145.991693,587.438707,-38.676880,8752812073
3,-5.466089,28.489701,279.301964,222.320940,76.762579,15982501021353414,348.821004,162.723413,56.858169,12.518181,1,35.499721,16.376210,148.953503,108.903666,-38.676676,8750045581
4,-65.586734,23.782355,128.880600,73.148219,73.401924,15982501021353419,54.891883,-6.050043,56.889778,12.093521,1,34.342534,13.494841,135.012985,33.328147,-38.676430,8751841300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11116201,NaN,NaN,0.000000,5.017072,69.755711,17711783703784922,120.353329,0.000000,58.390577,11.847116,1,NaN,12.534509,NaN,46.852902,-34.232391,5941110663
11116202,85.149412,22.113112,-1.573018,48.853637,69.902011,17711783703784923,44.586495,133.593291,58.390443,11.887254,1,32.060514,12.581314,142.156849,48.240569,-34.232552,5941110663
11116203,11.215623,22.103942,27.372585,21.041774,NaN,17711783703784924,0.000000,222.788788,58.390942,11.841447,1,32.054039,12.525000,140.849430,60.196099,-34.232415,5952753448
11116204,130.080544,35.409613,127.994723,79.995871,99.524767,17711783703784925,215.643587,6.311934,58.421559,16.643941,1,47.152890,19.900337,182.670056,139.175022,-34.222802,5939749788


In [29]:
gt_path = "/home/qiaozhih/bliss/case_studies/redshift/redshift_from_img/DC2_generate_catalog_output/merged_catalog_with_flux_over_50.pkl"
gt_catalog_df = pd.read_pickle(gt_path)

In [30]:
gt_catalog_df.head()

,cosmodc2_id,id,flux_i,flux_r,dec,match_objectId,flux_u,flux_g,flux_y,ra,...,IxxPSF_pixel_i,IyyPSF_pixel_g,IxyPSF_pixel_g,objectId,IxyPSF_pixel_y,psf_fwhm_i,IxyPSF_pixel_z,IxxPSF_pixel_r,IyyPSF_pixel_z,psf_fwhm_z
0,8757401501,8757401501,182.194626,144.206085,-38.677521,15982501021353368,123.476219,160.843887,263.547424,56.851958,...,3.042581,3.090587,0.006627,1.598250e+16,-0.000988,0.819964,0.001858,3.108767,3.866399,0.926216
1,8750694182,8750694182,99.906235,59.634422,-38.677104,15982501021353393,10.705873,21.402449,125.482666,56.799616,...,3.046197,3.213237,-0.003806,1.598250e+16,-0.003311,0.820432,0.001210,3.145637,4.014515,0.943202
2,8752812073,8752812073,503.124298,499.428711,-38.676882,15982501021353397,594.172546,550.806763,577.899048,56.883238,...,3.037378,3.067176,0.006382,1.598250e+16,-0.000901,0.819285,0.009736,3.135766,3.903795,0.931271
3,8750045581,8750045581,266.654419,216.263962,-38.676674,15982501021353414,51.661514,117.508293,336.939270,56.858170,...,3.042726,3.061446,0.008114,1.598250e+16,-0.002844,0.819805,0.003223,3.072602,3.884826,0.928279
4,8751841300,8751841300,127.011505,56.280338,-38.676423,15982501021353419,33.597462,40.201370,148.962173,56.889715,...,3.037196,3.022018,0.005205,1.598250e+16,-0.007553,0.819277,0.001210,3.111783,4.014515,0.943202


In [31]:
gt_test_df = gt_catalog_df[gt_catalog_df.flux_r < 200].sample(10000)

In [32]:
gt_test_df.columns

Index(['cosmodc2_id', 'id', 'flux_i', 'flux_r', 'dec', 'match_objectId',
       'flux_u', 'flux_g', 'flux_y', 'ra', 'truth_type', 'flux_z', 'redshifts',
       'size_bulge_true', 'position_angle_true', 'galaxy_id',
       'size_minor_disk_true', 'size_disk_true', 'size_minor_bulge_true',
       'bulge_to_total_ratio_i', 'psf_fwhm_g', 'psf_fwhm_r', 'IxxPSF_pixel_g',
       'psf_fwhm_y', 'IxyPSF_pixel_i', 'IyyPSF_pixel_i', 'IxyPSF_pixel_u',
       'IyyPSF_pixel_u', 'IxyPSF_pixel_r', 'IxxPSF_pixel_y', 'IxxPSF_pixel_z',
       'psf_fwhm_u', 'IxxPSF_pixel_u', 'IyyPSF_pixel_y', 'IyyPSF_pixel_r',
       'IxxPSF_pixel_i', 'IyyPSF_pixel_g', 'IxyPSF_pixel_g', 'objectId',
       'IxyPSF_pixel_y', 'psf_fwhm_i', 'IxyPSF_pixel_z', 'IxxPSF_pixel_r',
       'IyyPSF_pixel_z', 'psf_fwhm_z'],
      dtype='object')

In [62]:
lsst_test_df = lsst_catalog_df.merge(gt_test_df, left_on="id_truth", right_on="id")

In [64]:
column = ["id", "cModelFlux_u", "cModelFlux_g", "cModelFlux_r", "cModelFlux_i", "cModelFlux_z", "cModelFlux_y", "redshifts"]
lsst_test_df = lsst_test_df[column].dropna()
lsst_test_df = lsst_test_df[lsst_test_df.cModelFlux_r < 200]

In [ ]:
lsst_test_df = lsst_test_df[(lsst_test_df > 0).all(axis=1)]

In [81]:
survey_band = ["u", "g", "r", "i", "z", "y"]
flux_names = [f"cModelFlux_{band}" for band in survey_band]
lsst_flux_lists = [torch.tensor(lsst_test_df[flux_name].values, device=device) for flux_name in flux_names]
lsst_mag_lists = [22.5 - 2.5 * torch.log10(lsst_flux_list / 3631) for lsst_flux_list in lsst_flux_lists]
redshifts_gt = lsst_test_df.redshifts.values


In [83]:
lsst_mags = torch.stack(lsst_mag_lists, dim=-1)

In [84]:
reg.eval()
preds = reg.net(lsst_mags.float()).cpu().detach().numpy()
trues = redshifts_gt

In [ ]:
plt.xlim(0, 3)
plt.ylim(0, 3)
plt.axline((0, 0), slope=1)
plt.scatter(preds, trues, alpha=0.1, s=3)